# Object Detection with AWS: A Demo with Boots and Cats

This series of notebooks demonstrates tackling a sample computer vision problem on AWS - building a two-class object detector for [boots and cats](https://www.youtube.com/watch?v=Nni0rTLg5B8).

**This notebook** walks through using the [SageMaker Ground Truth](https://aws.amazon.com/sagemaker/groundtruth/) tool to annotate training and validation data sets.

**Follow-on** notebooks show how to train a range of models from the created dataset, including:

* [Amazon Rekognition](https://aws.amazon.com/rekognition/)'s new [custom labels](https://aws.amazon.com/rekognition/custom-labels-features/) functionality, announced at Re:Invent 2019
* SageMaker's [built-in object detection algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection.html)

# Boots 'n' Cats 1: Introduction and Data Preparation

## Acknowledgements

We use the [**Open Images Dataset v4**](https://storage.googleapis.com/openimages/web/download_v4.html) as a convenient source of pre-curated images. The Open Images Dataset V4 is created by Google Inc. We have not modified the images or the accompanying annotations. You can obtain the images and the annotations [here](https://storage.googleapis.com/openimages/web/download_v4.html). The annotations are licensed by Google Inc. under CC BY 4.0 license. The images are listed as having a CC BY 2.0 license. The following paper describes Open Images V4 in depth: from the data collection and annotation to detailed statistics about the data and evaluation of models trained on it.

A. Kuznetsova, H. Rom, N. Alldrin, J. Uijlings, I. Krasin, J. Pont-Tuset, S. Kamali, S. Popov, M. Malloci, T. Duerig, and V. Ferrari. The Open Images Dataset V4: Unified image classification, object detection, and visual relationship detection at scale. arXiv:1811.00982, 2018. ([link to PDF](https://arxiv.org/abs/1811.00982))

## Pre-requisites

This notebook is designed to be run in Amazon SageMaker. To run it (and understand what's going on), you'll need:

* Basic familiarity with Python, [AWS S3](https://docs.aws.amazon.com/s3/index.html), [Amazon Sagemaker](https://aws.amazon.com/sagemaker/), and the [AWS Command Line Interface (CLI)](https://aws.amazon.com/cli/).
* To run in **a region where Rekognition custom labelling is available** (Only N. Virginia at launch), if you plan to explore this feature.
* To create an **S3 bucket** in the same region, and ensure the SageMaker notebook's role has access to this bucket.
* Sufficient [SageMaker quota limits](https://docs.aws.amazon.com/general/latest/gr/aws_service_limits.html#limits_sagemaker) set on your account to run GPU-accelerated training jobs.

## Cost and runtime

Depending on your configuration, this demo may consume resources outside of the free tier but should not generally be expensive because we'll be training on a small number of images. You might wish to review the following for your region:

* [Amazon SageMaker pricing](https://aws.amazon.com/sagemaker/pricing/)
* [SageMaker Ground Truth pricing](https://aws.amazon.com/sagemaker/groundtruth/pricing/)
* [Amazon Rekognition pricing](https://aws.amazon.com/rekognition/pricing/)

The standard `ml.t2.medium` instance should be sufficient to run the notebooks.

We will use GPU-accelerated instance types for training and hyperparameter optimization, and use spot instances where appropriate to optimize these costs.

As noted in the step-by-step guidance, you should take particular care to delete any created SageMaker real-time prediction endpoints when finishing the demo.

## Step 0: Dependencies and configuration

As usual we'll start by loading libraries, defining configuration, and connecting to the AWS SDKs:

In [ ]:
%load_ext autoreload
%autoreload 1

# Built-Ins:
import csv
import os
from collections import defaultdict
import json

# External Dependencies:
import boto3
import imageio
import numpy as np
import sagemaker
from IPython.display import display, HTML

# Local Dependencies:
%aimport util

Next we configure the name and layout of your bucket, and the annotation job to set up.

**If you're following this demo in a group:**, you can pool your annotations for better accuracy without spending hours annotating:

* Have each group member set a different `BATCH_OFFSET` integer from 0 upwards and you'll be allocated different images to annotate
* Later, you can *import* the other members' output manifest files to your own S3 data set.

**If not:** don't worry - we already provide a 100-image set in this repository to augment your annotations!

In [ ]:
## Overall S3 bucket layout:
# Note you could instead set for auto-setup: BUCKET_NAME = sagemaker.Session().default_bucket()
BUCKET_NAME = # TODO: Put your bucket name here
%store BUCKET_NAME
DATA_PREFIX = "data" # The folder in the bucket (and locally) where we will store data
%store DATA_PREFIX
MODELS_PREFIX = "models" # The folder in the bucket where we will store models
%store MODELS_PREFIX
CHECKPOINTS_PREFIX = "models/checkpoints" # Model checkpoints can go in a subfolder of models
%store CHECKPOINTS_PREFIX

## Annotation job:
CLASS_NAMES = ["Boot", "Cat"]
%store CLASS_NAMES
N_EXAMPLES_PER_CLASS = 20
BATCH_OFFSET = 0
BATCH_NAME = "my-annotations"

# Note that some paths are reserved, restricting your choice of BATCH_NAME:
data_raw_prefix = DATA_PREFIX + "/raw"
data_augment_prefix = DATA_PREFIX + "/augmentation"
data_batch_prefix = f"{DATA_PREFIX}/{BATCH_NAME}"
test_image_folder = DATA_PREFIX + "/test"
%store test_image_folder

Here we just connect to the AWS SDKs we'll use, and validate the choice of S3 bucket:

In [ ]:
role = sagemaker.get_execution_role()
session = boto3.session.Session()
region = session.region_name
s3 = session.resource("s3")
bucket = s3.Bucket(BUCKET_NAME)
smclient = session.client("sagemaker")

bucket_region = \
    session.client("s3").head_bucket(Bucket=BUCKET_NAME)["ResponseMetadata"]["HTTPHeaders"]["x-amz-bucket-region"]
assert (
    bucket_region == region
), f"Your S3 bucket {BUCKET_NAME} and this notebook need to be in the same region."

if (region != "us-east-1"):
    print("WARNING: Rekognition Custom Labels functionality is only available in us-east-1 at launch")

## Step 1: Set the goalposts with some unlabelled target data

Let's start out by collecting a handful of images from around the web to illustrate what we'd like to detect.

These images are not licensed and the links may break for different regions / times in future: Feel free to add your own or replace with any other images of boots and cats!

Model evaluations in following notebooks will loop through each image in the `test_image_folder`

In [ ]:
!mkdir -p $test_image_folder
!wget -O $test_image_folder/tabby.jpg https://images.fineartamerica.com/images-medium-large-5/1990s-ginger-and-white-tabby-cat-animal-images.jpg
!wget -O $test_image_folder/beatbox.jpg https://midnightmusic.com.au/wp-content/uploads/2014/08/How-to-beatbox-5001.png
!wget -O $test_image_folder/ampersand.jpg https://i.ytimg.com/vi/DsC5hNYpP9Y/maxresdefault.jpg
!wget -O $test_image_folder/boots.jpg https://d28m5bx785ox17.cloudfront.net/v1/img/w4r1gr5IKcC9tTcJG_vsJVbyjZ_SVKuFf3YBxtrGdFs=/d/l
!wget -O $test_image_folder/cats.jpg https://www.dw.com/image/42582511_401.jpg

for test_image in os.listdir(test_image_folder):
    display(HTML(f"<h4>{test_image}</h4>"))
    util.visualize_detection(f"{test_image_folder}/{test_image}", [], [])

## Step 2: Map our class names to OpenImages class IDs

OpenImages defines a hierarchy of object types (e.g. "swan" is a subtype of "bird"), and references each with a class ID instead of the human-readable name.

Since we want to find images containing boots and cats, our first job is to figure what OpenImages class IDs they correspond to.

We start by downloading the OpenImages metadata, below.

(Note we're only referencing the `test` subset of OpenImages as an easy way to keep data volumes small)

In [ ]:
# Download and process the Open Images annotations.
os.makedirs(data_raw_prefix, exist_ok=True)
!wget -O $data_raw_prefix/annotations-bbox.csv https://storage.googleapis.com/openimages/2018_04/test/test-annotations-bbox.csv 
!wget -O $data_raw_prefix/class-descriptions.csv https://storage.googleapis.com/openimages/2018_04/class-descriptions.csv 
!wget -O $data_raw_prefix/labels-hierarchy.json https://storage.googleapis.com/openimages/2018_04/bbox_labels_600_hierarchy.json

Now a case-insensitive lookup for our class names in the classes CSV:

In [ ]:
# The class list is really long, so let's stream it instead of loading dataframe:
class_root_ids = { s: None for s in CLASS_NAMES }
classes_lower_notfound = { s.lower(): s for s in CLASS_NAMES }
with open(f"{data_raw_prefix}/class-descriptions.csv", "r") as f:
    for row in csv.reader(f):
        row_class_lower = row[1].lower()
        match = classes_lower_notfound.get(row_class_lower)
        if (match is not None):
            class_root_ids[match] = row[0]
            del classes_lower_notfound[row_class_lower]
            if (len(classes_lower_notfound) == 0):
                print("Class name -> root ID mapping done")
                break

print(class_root_ids)
if len(classes_lower_notfound):
    raise ValueError(
        f"IDs not found for these class names: {[v for (k,v) in classes_lower_notfound.items()]}"
    )

Next, we recurse down the ontology from these root classes to capture any child classes.

(Note that actually "boot" and "cat" are leaf nodes in OpenImages v4, but other common demos like "bird" are not).

In [ ]:
with open(f"{data_raw_prefix}/labels-hierarchy.json", "r") as f:
    hierarchy = json.load(f)

def get_all_subclasses(class_id, tree):
    """Get the set of `class_id` and all matching subclasses from hierarchy `tree`"""
    def all_subtree_class_ids(subtree):
        if ("Subcategory" in subtree):
            return set([subtree["LabelName"]]).union(
                *[all_subtree_class_ids(s) for s in subtree["Subcategory"]]
            )
        else:
            return set([subtree["LabelName"]])
    if (tree["LabelName"] == class_id):
        return all_subtree_class_ids(tree)
    elif "Subcategory" in tree:
        return set().union(*[get_all_subclasses(class_id, s) for s in tree["Subcategory"]])
    else:
        return set()

class_id_sets = {
    name: get_all_subclasses(class_root_ids[name], hierarchy) for name in class_root_ids
}
print("Final OpenImages class ID sets:")
print(class_id_sets)

## Step 3: Find suitable example images

Now we've looked up the full range of applicable label IDs, we can use the OpenImages annotations to extract which image IDs will be interesting for us to train on (i.e. they contain boots and/or cats).

We deliberately search through the data-set in deterministic order, and only want to collect `N_EXAMPLES_PER_CLASS` images for each label but need to offset the ones we pick up by `BATCH_OFFSET` if this is non-zero.

In [ ]:
# Skip these images with known bad quality content:
SKIP_IMAGES = {"251d4c429f6f9c39", "065ad49f98157c8d"}

# Dict[class_name][img_id] -> [class_name, xmin, xmax, ymin, ymax]
class_bbs = { name: defaultdict(list) for name in class_id_sets }

# BATCH_OFFSET allows 
n_images_needed = N_EXAMPLES_PER_CLASS * (BATCH_OFFSET + 1)

unfilled_class_names = set(CLASS_NAMES)
with open(f"{data_raw_prefix}/annotations-bbox.csv", "r") as f:
    for row in csv.reader(f):
        img_id, _, cls_id, conf, xmin, xmax, ymin, ymax, *_ = row
        if (img_id in SKIP_IMAGES):
            continue
        curr_unfilled_class_names = unfilled_class_names.copy()
        for name in curr_unfilled_class_names:
            if (cls_id in class_id_sets[name]):
                class_bbs[name][img_id].append([name, xmin, xmax, ymin, ymax])
                if (len(class_bbs[name]) >= n_images_needed):
                    unfilled_class_names.remove(name)
                    
if (len(unfilled_class_names)):
    print(
        "WARNING: Found fewer than ("
        + f"{N_EXAMPLES_PER_CLASS}x{BATCH_OFFSET+1}={n_images_needed}"
        + ") requested images for the following classes:\n"
        + "\n".join([f"{name} ({len(class_bbs[name])} images)" for name in unfilled_class_names])
    )

bbs = defaultdict(list)
for class_name in class_bbs:
    # Take last N_EXAMPLES_PER_CLASS images from each class (for BATCH_OFFSET)
    class_bbs_all_unfiltered = list(class_bbs[class_name].items())
    class_bbs_batch = class_bbs_all_unfiltered[-N_EXAMPLES_PER_CLASS:]
    class_bbs[class_name] = defaultdict(list, class_bbs_batch)
    # Concatenate each class together into the overall `bbs` set
    for (img_id, boxes) in class_bbs_batch:
        bbs[img_id] = bbs[img_id] + boxes

image_ids = bbs.keys()
n_images = len(image_ids)
print(f"Selected {n_images} images")

## Step 4: Upload images and manifest file to S3

We need our training image data in an accessible S3 bucket, and a [manifest](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-data-input.html) file defining for SageMaker Ground Truth (and later our model) what images are in the data set and where to find them.

In the following cell, we:

* Copy each identified image directly from the OpenImages repository to our bucket
* Build up a local manifest file listing all the images
* Upload the manifest file to the bucket

This process should only take a few seconds with small data sets like we're dealing with here.

In [ ]:
os.makedirs(f"{data_batch_prefix}/manifests", exist_ok=True)
input_manifest_loc = f"{data_batch_prefix}/manifests/input.manifest"

with open(input_manifest_loc, "w") as f:
    print("Copying images", end="")
    # TODO: Delete existing folder contents?
    for image_id in image_ids:
        print(".", end="")
        dest_key = f"{data_batch_prefix}/images/{image_id}.jpg"
        bucket.copy(
            {
                "Bucket": "open-images-dataset",
                "Key": f"test/{image_id}.jpg"
            },
            dest_key
        )
        f.write(json.dumps({ "source-ref": f"s3://{BUCKET_NAME}/{dest_key}" }) + "\n")
    print("")
    print(f"Images copied to s3://{BUCKET_NAME}/{data_batch_prefix}/images/")

bucket.upload_file(input_manifest_loc, input_manifest_loc)
print(f"Manifest uploaded to s3://{BUCKET_NAME}/{input_manifest_loc}")

## Step 5: Set up the SageMaker Ground Truth labelling job

Now that our images and a manifest file listing them are ready in S3, we'll set up the Ground Truth labelling job **in the [AWS console](https://console.aws.amazon.com)**.

Under *Services* go to *Amazon SageMaker*, and select *Ground Truth > Labeling Jobs* from the side-bar menu on the left.

**Note:** These steps assume you've either never used SageMaker Ground Truth before, or have already set up a Private Workforce that will be suitable for this task. If you have one or more private workforces configured already, but none of them are appropriate for this task, you'll need to go to *Ground Truth > Labeling workforces* **first** to create a new one.

### Job Details

Click the **Create labeling job** button, and you'll be asked to specify job details as follows:

* **Job name:** Choose a name to identify this labelling job, e.g. `boots-and-cats-batch-0`
* **Label name (The override checkbox):** Consider overriding this to `labels`
* **Input data location:** The path to the input manifest file in S3 (see output above)
* **Output data location:** Set this just to the parent folder of the input manifests (e.g. *s3://gt-object-detect-thewsey-us-east-1/data/my-annotations*)
* **IAM role:** If you're not sure whether your existing roles have the sufficient permissions for Ground Truth, select the options to create a new role
* **Task type:** Image > Bounding box

<img src="BlogImages/JobDetailsIntro.png"/>

All other settings can be left as default. Record your choices for the label name and output data location below, because we'll need these later:

In [ ]:
my_groundtruth_job_name = # TODO: "boots-and-cats-batch-0"?
my_groundtruth_output = f"s3://{BUCKET_NAME}/data/my-annotations" # TODO: **No trailing slash!**
my_groundtruth_labels = "labels" # TODO: Check this matches yours

### Workers

On the next screen, we'll configure **who** will annotate our data: Ground Truth allows you to define your own in-house *Private Workforces*; use *Vendor Managed Workforces* for specialist tasks; or use the public workforce provided by *Amazon Mechanical Turk*.

Select **Private** worker type, and you'll be prompted either to select from your existing private workforces, or create a new one if none exist.

To create a new private workforce if you need, simply follow the UI workflow with default settings. It doesn't matter what you call the workforce, and you can create a new Cognito User Group to define the workforce. **Add yourself** to the user pool by adding your email address: You should receive a confirmation email shortly with a temporary password and a link to access the annotation portal.

Automatic data labeling is applicable only for data sets over 1000 samples, so leave this turned **off** for now.

<img src="BlogImages/SelectPrivateWorkforce.png"/>

### Labeling Tool

Since you'll be labelling the data yourself, a brief description of the task should be fine in this case. When using real workforces, it's important to be really clear in this section about the task requirements and best practices - to ensure consistency of annotations between human workers.

For example: In the common case where we see a *pair* of boots from the side and one is almost entirely obscured, how should the image be annotated? Should *model* cats count, or only real ones?

The most important configuration here is to set the *options* to be the same as our `CLASS_NAMES` and in the same order: **Boot, Cat**

<img src="BlogImages/LabellingToolSetup.png"/>

Take some time to explore the other options for configuring the annotation tool; and when you're ready click "Create" to launch the labeling job.

## Step 6: Label those images!

Follow the link you received in your workforce invitation email to the workforce's **labelling portal**, and log in with the default password given in the email (which you'll be asked to change).

If you lose the portal link, you can always retrieve it through the *Ground Truth > Labeling Workforces* menu in the SageMaker console: Near the top of the summary of private workforces.

New jobs can sometimes take a minute or two to appear for workers, but you should soon see a screen like the below. Select the job and click "Start working" to enter the labelling tool.

<img src="BlogImages/LabellingJobsReady.png"/>

Note that you can check on the progress of labelling jobs through the APIs as well as in the AWS console:

In [ ]:
smclient.describe_labeling_job(LabelingJobName=my_groundtruth_job_name)['LabelingJobStatus']

Label all the images in the tool by selecting the class and drawing boxes around the objects, and when done you will be brought back to the (now empty) jobs list screen above.

It may take a few seconds after completing for the job status to update in the AWS console.

## Step 7: (Optional) Augment the labelled data

This repository contains an example output manifest (100 images) which we can use to augment our data set and improve our model's accuracy.

The script below **imports** the data to our bucket, since we don't have access to the S3 bucket where the annotation job was originally performed. We extract the OpenImages image ID/filename from each entry in the manifest; copy this image into our bucket; and create an updated output manifest with the new `source-ref`s.

The code includes some tips in case you want to re-purpose it for other data sources

In [ ]:
# Utility function definition:
def import_manifest_file(manifest_file_path, updated_manifest_file_path, target_prefix):
    """Import a Ground Truth output.manifest file to the current S3 bucket
    Parameters:
    ----------
    manifest_file_path: str
        Local relative path to an output manifest which was created against a different S3 bucket
    updated_manifest_file_path: str
        Local relative path (and S3 key) to store the new manifest with updated S3 keys
    target_prefix: str
        Place in current S3 bucket to store the images and updated manifest
    """
    with open(manifest_file_path, "r") as f_source:
        with open(updated_manifest_file_path, "w") as f_target:
            print("Copying images", end="")
            for line in f_source:
                print(".", end="")
                datum = json.loads(line)
                image_file = datum["source-ref"].rpartition("/")[2]
                dest_key = f"{target_prefix}/images/{image_file}"
                bucket.copy(
                    {
                        "Bucket": "open-images-dataset",
                        "Key": f"test/{image_file}"
                    },
                    dest_key
                )
                datum["source-ref"] = f"s3://{BUCKET_NAME}/{dest_key}"
                f_target.write(json.dumps(datum) + "\n")
    print("")
    # Upload the updated manifest to S3 for our use in model fitting:
    bucket.upload_file(updated_manifest_file_path, updated_manifest_file_path)
    print(f"Augmentation manifest uploaded to:\ns3://{BUCKET_NAME}/{updated_manifest_file_path}")


# Import the augmentation data included in the repository:
augment_manifest_loc = f"{data_augment_prefix}/manifests/output/output.updated.manifest"
import_manifest_file(
    f"{data_augment_prefix}/manifests/output/output.manifest",
    augment_manifest_loc,
    data_augment_prefix
)

In [ ]:
# If wanting to import another manifest file:

# Ensure the folder creation, if you need to:
#os.makedirs("data/[SUBFOLDERNAME]/manifests/output", exist_ok=True)

# Download the output.manifest to our standard location:
#!aws s3 cp s3://WHEREVER-THE-OUTPUT.MANIFEST-LIVES ./data/[SUBFOLDERNAME]/manifests/output/output.manfiest
# (Or copy it some other way if you don't have access via S3)

# Import the file:
# import_manifest_file(
#     "data/[SUBFOLDERNAME]/manifests/output/output.manifest",
#     "data/[SUBFOLDERNAME]/manifests/output/output.updated.manifest",
#     "data/[SUBFOLDERNAME]",
# )

## Step 8: Consolidate the labeling results
Now that our own labelling job is complete and we've imported other data to augment our data-set, we'll consolidate the batches together into a single combined manifest file.

First, validate and adjust the below configuration for your data:

In [ ]:
annotation_jobs = {
    # The batch you just annotated:
    # (Comment out this entry if you skipped creating & completing the Ground Truth job)
    BATCH_NAME: {
        # The label name you selected in the console:
        # (GroundTruth will set this equal to the job name by default)
        "label_name": my_groundtruth_labels, # TODO
        # This may be different if you selected something else for the output path other than:
        # s3://{BUCKET}/{batch_data_prefix}
        "manifest": f"{my_groundtruth_output}/{my_groundtruth_job_name}/manifests/output/output.manifest"
    },
    
    # The augmentation batch provided in the repo:
    # (Comment out this entry if you skipped importing this)
    "augmentation": {
        "label_name": "labels",
        "manifest": f"s3://{BUCKET_NAME}/data/augmentation/manifests/output/output.updated.manifest"
    }
    
    # Did you import any other batches?
}

print(json.dumps(annotation_jobs, indent=2))

Read the contents from each manifest and consolidate in memory:

In [ ]:
annotation_job_ids = list(annotation_jobs.keys())

for job_id in annotation_jobs:
    manifest_bucket, _, manifest_path = annotation_jobs[job_id]["manifest"][len("s3://"):].partition("/")
    manifest_folder, _, manifest_file = manifest_path.rpartition("/")
    os.makedirs(manifest_folder, exist_ok=True)
    s3.Bucket(manifest_bucket).download_file(manifest_path, manifest_path)
    with open(manifest_path, "r") as f:
        data = [json.loads(line) for line in f.readlines()]
        # Standardize label names:
        for datum in data:
            datum["labels"] = datum.pop(annotation_jobs[job_id]["label_name"])
        annotation_jobs[job_id]["data"] = data
        print(f"{len(data)} samples from annotation set {job_id}")

combined_manifest_data = [datum for job_id in annotation_jobs for datum in annotation_jobs[job_id]["data"]]
print(f"Got {len(combined_manifest_data)} total samples from {len(annotation_job_ids)} sets")

# The standardization above means these are always the attributes training will care about:
attribute_names = ["source-ref", "labels"]
%store attribute_names

# For illustration, this is what an entry in our combined manifest looks like:
combined_manifest_data[0]

## Step 9: Split training vs validation and upload final manifests

Now we have all our consolidated label sets (and all the referenced images uploaded in our S3 bucket), the final step is to split training vs validation data and upload a manifest for each:

In [ ]:
# Shuffle output in place.
np.random.shuffle(combined_manifest_data)
    
n_samples_total = len(combined_manifest_data)
train_test_split_index = round(n_samples_total*0.8)

train_data = combined_manifest_data[:train_test_split_index]
validation_data = combined_manifest_data[train_test_split_index:]

n_samples_training = len(train_data)
%store n_samples_training
n_samples_validation = len(validation_data)
%store n_samples_validation

with open(f"{DATA_PREFIX}/train.manifest", "w") as f:
    for line in train_data:
        f.write(json.dumps(line))
        f.write("\n")
    
with open(f"{DATA_PREFIX}/validation.manifest", "w") as f:
    for line in validation_data:
        f.write(json.dumps(line))
        f.write("\n")
        
bucket.upload_file(f"{DATA_PREFIX}/train.manifest", f"{DATA_PREFIX}/train.manifest")
print("Training manifest uploaded to:\n" + f"s3://{BUCKET_NAME}/{DATA_PREFIX}/train.manifest")
bucket.upload_file(f"{DATA_PREFIX}/validation.manifest",f"{DATA_PREFIX}/validation.manifest")
print("Validation manifest uploaded to:\n" + f"s3://{BUCKET_NAME}/{DATA_PREFIX}/validation.manifest")

## Review

Phew! That felt like a lot of work, but a lot of the steps were hacks for our example:

* To find raw image data for our targets (boots and cats), we mapped our class names to the public OpenImages dataset and used their existing annotations to find relevant images.
* To get a decent data volume without spending forever annotating in the workshop, we merged our Ground Truth annotation results with other augmentation sets.

The useful points to remember are:

* SageMaker Ground Truth (and as we'll see later, many of the built-in algorithms as well) uses **augmented manifests** to define annotated image datasets.
* These manifests are just plain text [JSON Lines](http://jsonlines.org/) files that we can also edit in our own code to do whatever we like from importing/exporting annotations, to stitching together datasets as we did here.
* Once the input manifest is prepared, it only takes a few clicks to define workforce teams and annotation jobs in SageMaker Ground Truth: Which supports other built-in and even custom annotation workflows for a variety of data types and tasks.

Although we didn't use it here due to the dataset size, the [automated labelling](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-automated-labeling.html) feature can drastically cut annotation costs and time on bigger data-sets for the tasks where it's supported (including object detection).

Ground Truth supports validation workflows (typically much faster for humans) as well as labelling; which can be combined with automated labelling in light of the importance of good quality ground truth input to effective machine learning.

In the follow-on notebooks, we'll use the composite training and validation datasets we created here to fit a variety of models and compare their performance. Let's move on to notebook 2(a)!